# 🏋️ Biceps Curl Form Analysis Notebook

Bu notebook, **Mediapipe Pose** kullanarak biceps curl videolarından form analizi yapar ve **RandomForest** ile "form doğru mu yanlış mı?" sınıflandırması gerçekleştirir.

## İçindekiler
1. Kütüphanelerin Kurulumu ve Import
2. Google Drive Mount (Opsiyonel)
3. Dataset Yolu Konfigürasyonu
4. Yardımcı Fonksiyonlar
5. Feature Extraction Pipeline
6. Dataset Oluşturma
7. RandomForest Model Eğitimi
8. Değerlendirme ve Görselleştirme
9. Model Kaydetme
10. Tek Video ile Tahmin (Opsiyonel)

---

## 1. Kütüphanelerin Kurulumu ve Import

Gerekli kütüphaneleri kurup import ediyoruz.

In [ ]:
# Gerekli kütüphaneleri kur
!pip install mediapipe opencv-python numpy pandas scikit-learn tqdm matplotlib joblib -q

In [ ]:
# Import'lar
import os
import cv2
import numpy as np
import pandas as pd
import mediapipe as mp
import matplotlib.pyplot as plt
import joblib
import warnings

from tqdm import tqdm
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix
)

# Uyarıları kapat
warnings.filterwarnings('ignore')

# Mediapipe Pose modülünü başlat
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

print("✅ Tüm kütüphaneler başarıyla yüklendi!")
print(f"OpenCV version: {cv2.__version__}")
print(f"Mediapipe version: {mp.__version__}")

## 2. Google Drive Mount (Opsiyonel)

Eğer dataset'iniz Google Drive'da ise bu hücreyi çalıştırın.

In [ ]:
# Google Drive'ı bağla (Colab'da çalıştırın)
# Eğer dataset lokal dosya sisteminde ise bu hücreyi atlayabilirsiniz.

from google.colab import drive
drive.mount('/content/drive')

print("✅ Google Drive başarıyla bağlandı!")

## 3. Dataset Yolu Konfigürasyonu

Dataset'inizin kök dizinini burada ayarlayın.

**Beklenen klasör yapısı:**
```
dataset/
  train/
    true/    # iyi form biceps curl videoları
    false/   # kötü form biceps curl videoları
  test/
    true/
    false/
```

In [ ]:
# TODO: Dataset kök dizinini buraya girin
# Google Drive kullanıyorsanız: "/content/drive/MyDrive/dataset"
# Colab'a upload ettiyseniz: "/content/dataset"

DATA_ROOT = "/content/dataset"  # <-- Buraya kendi yolunuzu yazın

# Video işleme parametreleri
FRAME_SKIP = 2  # Her kaç frame'den birini işleyeceğimiz (performans için)
MIN_FRAMES = 10  # Minimum frame sayısı (daha az frame'li videolar atlanır)
MAX_FRAMES = None  # Maksimum frame sayısı (None = tümü)

# Desteklenen video formatları
VIDEO_EXTENSIONS = ('.mp4', '.mov', '.avi', '.mkv', '.webm', '.m4v')

# Dataset yapısını kontrol et
print(f"📁 Dataset kök dizini: {DATA_ROOT}")
print(f"🎬 Frame skip: {FRAME_SKIP}")
print(f"📊 Minimum frame sayısı: {MIN_FRAMES}")

# Dizinlerin varlığını kontrol et
for split in ['train', 'test']:
    for label in ['true', 'false']:
        path = os.path.join(DATA_ROOT, split, label)
        if os.path.exists(path):
            video_count = len([f for f in os.listdir(path) if f.lower().endswith(VIDEO_EXTENSIONS)])
            print(f"  ✅ {split}/{label}: {video_count} video")
        else:
            print(f"  ⚠️ {split}/{label}: Dizin bulunamadı!")

## 4. Yardımcı Fonksiyonlar

### 4.1 Açı Hesaplama Fonksiyonu

Üç nokta arasındaki açıyı hesaplar (2D ve 3D landmark'lar için).

In [ ]:
def calculate_angle(point_a, point_b, point_c):
    """
    Üç nokta arasındaki açıyı hesaplar.
    
    Açı, point_b noktasındaki açıdır (point_a -> point_b -> point_c).
    
    Args:
        point_a: İlk nokta (tuple veya array: x, y veya x, y, z)
        point_b: Köşe noktası (açı burada ölçülür)
        point_c: Son nokta
    
    Returns:
        float: Derece cinsinden açı (0-180)
    """
    a = np.array(point_a)
    b = np.array(point_b)
    c = np.array(point_c)
    
    # Vektörleri hesapla
    ba = a - b
    bc = c - b
    
    # Kosinüs formülü ile açı hesapla
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc) + 1e-8)
    
    # Sayısal hataları önlemek için -1 ile 1 arasında sınırla
    cosine_angle = np.clip(cosine_angle, -1.0, 1.0)
    
    # Radyandan dereceye çevir
    angle = np.degrees(np.arccos(cosine_angle))
    
    return angle


def calculate_vertical_angle(shoulder, hip):
    """
    Omuz-kalça hattının dikey eksene göre açısını hesaplar (torso angle).
    
    Args:
        shoulder: Omuz koordinatları (x, y)
        hip: Kalça koordinatları (x, y)
    
    Returns:
        float: Derece cinsinden torso açısı
    """
    # Dikey referans noktası (omuzun tam üstünde bir nokta)
    vertical_point = (shoulder[0], shoulder[1] - 1.0)  # Yukarı doğru
    
    return calculate_angle(vertical_point, shoulder, hip)


# Test
test_angle = calculate_angle((0, 0), (1, 1), (2, 0))
print(f"Test açısı (90 derece olmalı): {test_angle:.1f}°")

### 4.2 Video Listeleme Fonksiyonu

Train veya test klasöründeki tüm videoları label'ları ile birlikte listeler.

In [ ]:
def list_videos(split_dir):
    """
    Belirtilen split dizinindeki tüm videoları label'ları ile listeler.
    
    Args:
        split_dir: Split dizini (örn: /content/dataset/train)
    
    Returns:
        list: [(video_path, label), ...] formatında tuple listesi
              true klasörü -> label = 1 (iyi form)
              false klasörü -> label = 0 (kötü form)
    """
    videos = []
    
    # Label klasörlerini tara
    label_mapping = {'true': 1, 'false': 0}
    
    for label_name, label_value in label_mapping.items():
        label_dir = os.path.join(split_dir, label_name)
        
        if not os.path.exists(label_dir):
            print(f"⚠️ Uyarı: {label_dir} dizini bulunamadı!")
            continue
        
        # Video dosyalarını bul
        for filename in os.listdir(label_dir):
            if filename.lower().endswith(VIDEO_EXTENSIONS):
                video_path = os.path.join(label_dir, filename)
                videos.append((video_path, label_value))
    
    print(f"📹 {split_dir} dizininde {len(videos)} video bulundu.")
    
    return videos


# Test - sadece dizin varsa çalışacak
if os.path.exists(os.path.join(DATA_ROOT, 'train')):
    test_videos = list_videos(os.path.join(DATA_ROOT, 'train'))
    print(f"Örnek: {test_videos[:2] if test_videos else 'Video bulunamadı'}")

## 5. Feature Extraction Pipeline

### 5.1 Tek Video için Feature Çıkarma Fonksiyonu

Bu fonksiyon, bir videodan tüm biceps curl feature'larını çıkarır.

In [ ]:
def extract_features_from_video(video_path, pose, frame_skip=2, max_frames=None, min_frames=10):
    """
    Tek bir videodan biceps curl için feature'ları çıkarır.
    
    Args:
        video_path: Video dosyasının yolu
        pose: Mediapipe Pose nesnesi
        frame_skip: Kaç frame'de bir işleneceği (performans için)
        max_frames: İşlenecek maksimum frame sayısı (None = tümü)
        min_frames: Minimum frame sayısı (daha azsa None döner)
    
    Returns:
        dict: Feature dictionary veya None (hata durumunda)
    """
    
    # Video'yu aç
    cap = cv2.VideoCapture(video_path)
    
    if not cap.isOpened():
        print(f"⚠️ Video açılamadı: {video_path}")
        return None
    
    # Video özellikleri
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    if fps <= 0:
        fps = 30.0  # Varsayılan FPS
    
    # Frame bazlı ölçümler için listeler
    elbow_left_angles = []
    elbow_right_angles = []
    
    shoulder_left_x = []
    shoulder_left_y = []
    shoulder_right_x = []
    shoulder_right_y = []
    
    shoulder_left_angles = []  # Omuz açısı (dirsek-omuz-kalça)
    shoulder_right_angles = []
    
    torso_angles = []  # Gövde açısı (dikey eksene göre)
    
    frame_count = 0
    processed_frames = 0
    
    # Landmarklar için indeksler
    # Sol taraf
    LEFT_SHOULDER = mp_pose.PoseLandmark.LEFT_SHOULDER.value
    LEFT_ELBOW = mp_pose.PoseLandmark.LEFT_ELBOW.value
    LEFT_WRIST = mp_pose.PoseLandmark.LEFT_WRIST.value
    LEFT_HIP = mp_pose.PoseLandmark.LEFT_HIP.value
    
    # Sağ taraf
    RIGHT_SHOULDER = mp_pose.PoseLandmark.RIGHT_SHOULDER.value
    RIGHT_ELBOW = mp_pose.PoseLandmark.RIGHT_ELBOW.value
    RIGHT_WRIST = mp_pose.PoseLandmark.RIGHT_WRIST.value
    RIGHT_HIP = mp_pose.PoseLandmark.RIGHT_HIP.value
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        if not ret:
            break
        
        frame_count += 1
        
        # Frame skip uygula
        if frame_count % frame_skip != 0:
            continue
        
        # Maksimum frame kontrolü
        if max_frames and processed_frames >= max_frames:
            break
        
        # RGB'ye çevir (Mediapipe RGB bekler)
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Pose tespiti
        results = pose.process(frame_rgb)
        
        if results.pose_landmarks:
            landmarks = results.pose_landmarks.landmark
            
            # Landmark koordinatlarını al
            # Sol taraf
            l_shoulder = [landmarks[LEFT_SHOULDER].x, landmarks[LEFT_SHOULDER].y]
            l_elbow = [landmarks[LEFT_ELBOW].x, landmarks[LEFT_ELBOW].y]
            l_wrist = [landmarks[LEFT_WRIST].x, landmarks[LEFT_WRIST].y]
            l_hip = [landmarks[LEFT_HIP].x, landmarks[LEFT_HIP].y]
            
            # Sağ taraf
            r_shoulder = [landmarks[RIGHT_SHOULDER].x, landmarks[RIGHT_SHOULDER].y]
            r_elbow = [landmarks[RIGHT_ELBOW].x, landmarks[RIGHT_ELBOW].y]
            r_wrist = [landmarks[RIGHT_WRIST].x, landmarks[RIGHT_WRIST].y]
            r_hip = [landmarks[RIGHT_HIP].x, landmarks[RIGHT_HIP].y]
            
            # === DİRSEK AÇILARI ===
            # Sol dirsek açısı (omuz -> dirsek -> bilek)
            left_elbow_angle = calculate_angle(l_shoulder, l_elbow, l_wrist)
            elbow_left_angles.append(left_elbow_angle)
            
            # Sağ dirsek açısı
            right_elbow_angle = calculate_angle(r_shoulder, r_elbow, r_wrist)
            elbow_right_angles.append(right_elbow_angle)
            
            # === OMUZ KOORDİNATLARI ===
            shoulder_left_x.append(landmarks[LEFT_SHOULDER].x)
            shoulder_left_y.append(landmarks[LEFT_SHOULDER].y)
            shoulder_right_x.append(landmarks[RIGHT_SHOULDER].x)
            shoulder_right_y.append(landmarks[RIGHT_SHOULDER].y)
            
            # === OMUZ AÇILARI (dirsek-omuz-kalça) ===
            left_shoulder_angle = calculate_angle(l_elbow, l_shoulder, l_hip)
            shoulder_left_angles.append(left_shoulder_angle)
            
            right_shoulder_angle = calculate_angle(r_elbow, r_shoulder, r_hip)
            shoulder_right_angles.append(right_shoulder_angle)
            
            # === GÖVDE AÇISI ===
            # Orta omuz ve orta kalça noktaları
            mid_shoulder = [
                (l_shoulder[0] + r_shoulder[0]) / 2,
                (l_shoulder[1] + r_shoulder[1]) / 2
            ]
            mid_hip = [
                (l_hip[0] + r_hip[0]) / 2,
                (l_hip[1] + r_hip[1]) / 2
            ]
            
            torso_angle = calculate_vertical_angle(mid_shoulder, mid_hip)
            torso_angles.append(torso_angle)
            
            processed_frames += 1
    
    cap.release()
    
    # Minimum frame kontrolü
    if processed_frames < min_frames:
        print(f"⚠️ Yetersiz frame ({processed_frames}): {video_path}")
        return None
    
    # === FEATURE'LARI HESAPLA ===
    features = {}
    
    # --- Video süresi ---
    features['video_duration'] = total_frames / fps
    features['processed_frames'] = processed_frames
    
    # --- Dirsek Açısı Feature'ları ---
    # Sol dirsek
    features['elbow_left_min'] = np.min(elbow_left_angles)
    features['elbow_left_max'] = np.max(elbow_left_angles)
    features['elbow_left_range'] = features['elbow_left_max'] - features['elbow_left_min']
    features['elbow_left_mean'] = np.mean(elbow_left_angles)
    features['elbow_left_std'] = np.std(elbow_left_angles)
    
    # Sağ dirsek
    features['elbow_right_min'] = np.min(elbow_right_angles)
    features['elbow_right_max'] = np.max(elbow_right_angles)
    features['elbow_right_range'] = features['elbow_right_max'] - features['elbow_right_min']
    features['elbow_right_mean'] = np.mean(elbow_right_angles)
    features['elbow_right_std'] = np.std(elbow_right_angles)
    
    # Dirsek simetrisi
    elbow_diffs = np.abs(np.array(elbow_left_angles) - np.array(elbow_right_angles))
    features['elbow_lr_diff_mean'] = np.mean(elbow_diffs)
    features['elbow_lr_diff_max'] = np.max(elbow_diffs)
    
    # --- Omuz Stabilitesi Feature'ları ---
    features['shoulder_left_x_std'] = np.std(shoulder_left_x)
    features['shoulder_left_y_std'] = np.std(shoulder_left_y)
    features['shoulder_right_x_std'] = np.std(shoulder_right_x)
    features['shoulder_right_y_std'] = np.std(shoulder_right_y)
    
    # Omuz açısı feature'ları
    features['shoulder_left_angle_min'] = np.min(shoulder_left_angles)
    features['shoulder_left_angle_max'] = np.max(shoulder_left_angles)
    features['shoulder_left_angle_range'] = features['shoulder_left_angle_max'] - features['shoulder_left_angle_min']
    features['shoulder_left_angle_std'] = np.std(shoulder_left_angles)
    
    features['shoulder_right_angle_min'] = np.min(shoulder_right_angles)
    features['shoulder_right_angle_max'] = np.max(shoulder_right_angles)
    features['shoulder_right_angle_range'] = features['shoulder_right_angle_max'] - features['shoulder_right_angle_min']
    features['shoulder_right_angle_std'] = np.std(shoulder_right_angles)
    
    # Omuz simetrisi
    shoulder_angle_diffs = np.abs(np.array(shoulder_left_angles) - np.array(shoulder_right_angles))
    features['shoulder_lr_diff_mean'] = np.mean(shoulder_angle_diffs)
    
    # --- Gövde (Torso) Stabilitesi Feature'ları ---
    features['torso_angle_mean'] = np.mean(torso_angles)
    features['torso_angle_std'] = np.std(torso_angles)
    features['torso_angle_range'] = np.max(torso_angles) - np.min(torso_angles)
    features['torso_angle_min'] = np.min(torso_angles)
    features['torso_angle_max'] = np.max(torso_angles)
    
    # --- Açısal Hız Feature'ları (Kinematik) ---
    # Sol dirsek açısal hız (frame bazlı türev)
    elbow_left_vel = np.diff(elbow_left_angles)
    features['elbow_left_ang_vel_max'] = np.max(np.abs(elbow_left_vel)) if len(elbow_left_vel) > 0 else 0
    features['elbow_left_ang_vel_std'] = np.std(elbow_left_vel) if len(elbow_left_vel) > 0 else 0
    features['elbow_left_ang_vel_mean'] = np.mean(np.abs(elbow_left_vel)) if len(elbow_left_vel) > 0 else 0
    
    # Sağ dirsek açısal hız
    elbow_right_vel = np.diff(elbow_right_angles)
    features['elbow_right_ang_vel_max'] = np.max(np.abs(elbow_right_vel)) if len(elbow_right_vel) > 0 else 0
    features['elbow_right_ang_vel_std'] = np.std(elbow_right_vel) if len(elbow_right_vel) > 0 else 0
    features['elbow_right_ang_vel_mean'] = np.mean(np.abs(elbow_right_vel)) if len(elbow_right_vel) > 0 else 0
    
    return features


print("✅ Feature extraction fonksiyonu tanımlandı!")

### 5.2 Dataset Oluşturma Fonksiyonu

Tüm videoları işleyip feature DataFrame'i oluşturur.

In [ ]:
def build_dataset(split_dir, frame_skip=2, max_frames=None, min_frames=10):
    """
    Belirtilen split dizinindeki tüm videolardan feature'ları çıkarır.
    
    Args:
        split_dir: Split dizini (örn: /content/dataset/train)
        frame_skip: Kaç frame'de bir işleneceği
        max_frames: İşlenecek maksimum frame sayısı
        min_frames: Minimum frame sayısı
    
    Returns:
        X: Feature DataFrame
        y: Label Series
        video_paths: Video yolları listesi
    """
    
    # Video listesini al
    videos = list_videos(split_dir)
    
    if len(videos) == 0:
        print(f"⚠️ Hiç video bulunamadı: {split_dir}")
        return None, None, None
    
    all_features = []
    all_labels = []
    all_paths = []
    
    # Mediapipe Pose başlat
    with mp_pose.Pose(
        static_image_mode=False,
        model_complexity=1,
        smooth_landmarks=True,
        enable_segmentation=False,
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5
    ) as pose:
        
        # Her video için feature çıkar
        for video_path, label in tqdm(videos, desc=f"Processing {os.path.basename(split_dir)}"):
            
            features = extract_features_from_video(
                video_path=video_path,
                pose=pose,
                frame_skip=frame_skip,
                max_frames=max_frames,
                min_frames=min_frames
            )
            
            if features is not None:
                all_features.append(features)
                all_labels.append(label)
                all_paths.append(video_path)
    
    if len(all_features) == 0:
        print(f"⚠️ Geçerli feature çıkarılamadı: {split_dir}")
        return None, None, None
    
    # DataFrame oluştur
    X = pd.DataFrame(all_features)
    y = pd.Series(all_labels, name='label')
    
    print(f"\n✅ {split_dir} dataset oluşturuldu:")
    print(f"   Toplam video: {len(all_features)}")
    print(f"   Feature sayısı: {X.shape[1]}")
    print(f"   Label dağılımı: True={sum(y==1)}, False={sum(y==0)}")
    
    return X, y, all_paths


print("✅ Dataset oluşturma fonksiyonu tanımlandı!")

## 6. Dataset Oluşturma

Train ve test set'lerini oluşturuyoruz.

In [ ]:
# Train dataset oluştur
print("="*60)
print("📊 TRAIN DATASET OLUŞTURULUYOR...")
print("="*60)

X_train, y_train, train_paths = build_dataset(
    split_dir=os.path.join(DATA_ROOT, "train"),
    frame_skip=FRAME_SKIP,
    max_frames=MAX_FRAMES,
    min_frames=MIN_FRAMES
)

In [ ]:
# Test dataset oluştur
print("="*60)
print("📊 TEST DATASET OLUŞTURULUYOR...")
print("="*60)

X_test, y_test, test_paths = build_dataset(
    split_dir=os.path.join(DATA_ROOT, "test"),
    frame_skip=FRAME_SKIP,
    max_frames=MAX_FRAMES,
    min_frames=MIN_FRAMES
)

In [ ]:
# Feature'ları incele
if X_train is not None:
    print("\n📋 TRAIN DATASET ÖZET:")
    print(f"Shape: {X_train.shape}")
    print(f"\nFeature'lar:")
    for i, col in enumerate(X_train.columns, 1):
        print(f"  {i:2d}. {col}")
    
    print(f"\n📊 İlk 5 satır:")
    display(X_train.head())
else:
    print("⚠️ Train dataset oluşturulamadı!")

## 7. RandomForest Model Eğitimi

Model tanımı ve eğitimi.

In [ ]:
# Null kontrolü
if X_train is None or X_test is None:
    raise ValueError("❌ Dataset oluşturulamadı! Lütfen DATA_ROOT yolunu ve video dosyalarını kontrol edin.")

# NaN değerleri kontrol et ve temizle
if X_train.isnull().any().any():
    print("⚠️ Train set'te NaN değerler var, temizleniyor...")
    X_train = X_train.fillna(0)

if X_test.isnull().any().any():
    print("⚠️ Test set'te NaN değerler var, temizleniyor...")
    X_test = X_test.fillna(0)

print(f"\n✅ Train set: {X_train.shape[0]} örnek, {X_train.shape[1]} feature")
print(f"✅ Test set: {X_test.shape[0]} örnek, {X_test.shape[1]} feature")

In [ ]:
# RandomForest modeli oluştur
print("🌳 RandomForest modeli oluşturuluyor...")

rf_model = RandomForestClassifier(
    n_estimators=200,           # Ağaç sayısı
    max_depth=None,             # Maksimum derinlik (None = sınırsız)
    min_samples_split=2,        # Bölünme için minimum örnek sayısı
    min_samples_leaf=1,         # Yaprakta minimum örnek sayısı
    max_features='sqrt',        # Her bölünmede kullanılacak feature sayısı
    class_weight='balanced',    # Dengesiz sınıflar için ağırlıklandırma
    random_state=42,            # Tekrarlanabilirlik için
    n_jobs=-1,                  # Tüm CPU çekirdeklerini kullan
    verbose=1
)

# Modeli eğit
print("\n🏋️ Model eğitiliyor...")
rf_model.fit(X_train, y_train)

print("\n✅ Model eğitimi tamamlandı!")

## 8. Değerlendirme ve Görselleştirme

Model performansını ölçüyoruz.

In [ ]:
# Tahmin yap
y_pred = rf_model.predict(X_test)

# Accuracy
accuracy = accuracy_score(y_test, y_pred)
print("="*60)
print("📊 MODEL DEĞERLENDİRME SONUÇLARI")
print("="*60)
print(f"\n🎯 Accuracy: {accuracy:.4f} ({accuracy*100:.2f}%)")

In [ ]:
# Classification Report
print("\n📋 CLASSIFICATION REPORT:")
print("-"*60)

target_names = ['Kötü Form (False)', 'İyi Form (True)']
print(classification_report(y_test, y_pred, target_names=target_names))

In [ ]:
# Confusion Matrix
print("\n📊 CONFUSION MATRIX:")
print("-"*60)

cm = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(
    cm,
    index=['Gerçek: Kötü Form', 'Gerçek: İyi Form'],
    columns=['Tahmin: Kötü Form', 'Tahmin: İyi Form']
)

display(cm_df)

# Confusion Matrix görselleştirme
plt.figure(figsize=(8, 6))
plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Confusion Matrix', fontsize=14, fontweight='bold')
plt.colorbar()

tick_marks = np.arange(2)
plt.xticks(tick_marks, ['Kötü Form\n(Predicted)', 'İyi Form\n(Predicted)'], fontsize=11)
plt.yticks(tick_marks, ['Kötü Form\n(Actual)', 'İyi Form\n(Actual)'], fontsize=11)

# Değerleri hücrelere yaz
thresh = cm.max() / 2.
for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
        plt.text(j, i, format(cm[i, j], 'd'),
                 ha="center", va="center",
                 color="white" if cm[i, j] > thresh else "black",
                 fontsize=16, fontweight='bold')

plt.ylabel('Gerçek Label', fontsize=12)
plt.xlabel('Tahmin Edilen Label', fontsize=12)
plt.tight_layout()
plt.show()

In [ ]:
# Feature Importance
print("\n📊 FEATURE IMPORTANCE (İlk 15):")
print("-"*60)

# Feature importance'ları al
feature_importance = pd.DataFrame({
    'feature': X_train.columns,
    'importance': rf_model.feature_importances_
}).sort_values('importance', ascending=False)

# İlk 15 feature'ı göster
top_features = feature_importance.head(15)
print(top_features.to_string(index=False))

# Görselleştirme
plt.figure(figsize=(12, 8))
colors = plt.cm.RdYlGn(np.linspace(0.2, 0.8, len(top_features)))
plt.barh(range(len(top_features)), top_features['importance'].values, color=colors)
plt.yticks(range(len(top_features)), top_features['feature'].values, fontsize=11)
plt.xlabel('Feature Importance', fontsize=12)
plt.title('Top 15 Feature Importance\n(Biceps Curl Form Analysis)', fontsize=14, fontweight='bold')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

In [ ]:
# Train set performansı (overfitting kontrolü)
print("\n📊 OVERFITTING KONTROLÜ:")
print("-"*60)

y_train_pred = rf_model.predict(X_train)
train_accuracy = accuracy_score(y_train, y_train_pred)

print(f"Train Accuracy: {train_accuracy:.4f} ({train_accuracy*100:.2f}%)")
print(f"Test Accuracy:  {accuracy:.4f} ({accuracy*100:.2f}%)")
print(f"Fark:           {(train_accuracy - accuracy):.4f}")

if train_accuracy - accuracy > 0.1:
    print("\n⚠️ Overfitting olabilir! Model regularization veya daha fazla veri gerekebilir.")
else:
    print("\n✅ Model iyi genelleme yapıyor gibi görünüyor.")

## 9. Model Kaydetme

Eğitilmiş modeli dosyaya kaydediyoruz.

In [ ]:
# Model kaydetme
MODEL_PATH = "biceps_curl_rf_model.joblib"

joblib.dump(rf_model, MODEL_PATH)
print(f"✅ Model kaydedildi: {MODEL_PATH}")

# Feature listesini de kaydet (inference için gerekli)
FEATURE_LIST_PATH = "biceps_curl_features.txt"
with open(FEATURE_LIST_PATH, 'w') as f:
    for feature in X_train.columns:
        f.write(f"{feature}\n")

print(f"✅ Feature listesi kaydedildi: {FEATURE_LIST_PATH}")

# Model boyutunu göster
import os
model_size = os.path.getsize(MODEL_PATH) / (1024 * 1024)
print(f"📦 Model boyutu: {model_size:.2f} MB")

In [ ]:
# Google Drive'a model kopyalama (opsiyonel)
# Bu hücreyi çalıştırmadan önce Drive'ı mount ettiğinizden emin olun

# TODO: İstediğiniz Drive yolunu ayarlayın
DRIVE_MODEL_PATH = "/content/drive/MyDrive/models/biceps_curl_rf_model.joblib"

try:
    import shutil
    os.makedirs(os.path.dirname(DRIVE_MODEL_PATH), exist_ok=True)
    shutil.copy(MODEL_PATH, DRIVE_MODEL_PATH)
    print(f"✅ Model Google Drive'a kopyalandı: {DRIVE_MODEL_PATH}")
except Exception as e:
    print(f"⚠️ Drive'a kopyalama başarısız: {e}")

## 10. Tek Video ile Tahmin (Opsiyonel)

Yeni bir video üzerinde model tahmini yapmak için kullanılabilir.

In [ ]:
def predict_on_single_video(video_path, model, feature_columns, frame_skip=2):
    """
    Tek bir video üzerinde form tahmini yapar.
    
    Args:
        video_path: Video dosyasının yolu
        model: Eğitilmiş RandomForest modeli
        feature_columns: Feature sıralaması (list)
        frame_skip: Kaç frame'de bir işleneceği
    
    Returns:
        dict: Tahmin sonuçları
    """
    
    print(f"\n🎬 Video analiz ediliyor: {os.path.basename(video_path)}")
    print("-"*50)
    
    # Feature çıkar
    with mp_pose.Pose(
        static_image_mode=False,
        model_complexity=1,
        smooth_landmarks=True,
        enable_segmentation=False,
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5
    ) as pose:
        
        features = extract_features_from_video(
            video_path=video_path,
            pose=pose,
            frame_skip=frame_skip,
            max_frames=None,
            min_frames=5
        )
    
    if features is None:
        print("❌ Feature çıkarılamadı!")
        return None
    
    # DataFrame'e çevir (model beklediği sırada)
    X = pd.DataFrame([features])[feature_columns]
    
    # NaN kontrolü
    X = X.fillna(0)
    
    # Tahmin yap
    prediction = model.predict(X)[0]
    probabilities = model.predict_proba(X)[0]
    
    # Sonuçları yazdır
    result = {
        'prediction': prediction,
        'label': 'İyi Form (True)' if prediction == 1 else 'Kötü Form (False)',
        'confidence_good': probabilities[1],
        'confidence_bad': probabilities[0],
        'features': features
    }
    
    print(f"\n📊 TAHMİN SONUCU:")
    print(f"   ➤ Form: {result['label']}")
    print(f"   ➤ İyi Form Olasılığı: {result['confidence_good']*100:.1f}%")
    print(f"   ➤ Kötü Form Olasılığı: {result['confidence_bad']*100:.1f}%")
    
    # Önemli metrikler
    print(f"\n📈 ÖNEMLİ METRİKLER:")
    print(f"   ➤ Sol Dirsek ROM: {features['elbow_left_range']:.1f}°")
    print(f"   ➤ Sağ Dirsek ROM: {features['elbow_right_range']:.1f}°")
    print(f"   ➤ Torso Açısı (ort): {features['torso_angle_mean']:.1f}°")
    print(f"   ➤ Omuz Stabilitesi (std): {features['shoulder_left_y_std']:.4f}")
    
    return result


print("✅ Tek video tahmin fonksiyonu tanımlandı!")

In [ ]:
# Örnek kullanım: Tek video üzerinde tahmin
# TODO: Test etmek istediğiniz video yolunu buraya girin

# SAMPLE_VIDEO_PATH = "/content/test_video.mp4"
# result = predict_on_single_video(
#     video_path=SAMPLE_VIDEO_PATH,
#     model=rf_model,
#     feature_columns=X_train.columns.tolist(),
#     frame_skip=2
# )

print("💡 Tek video tahmini için yukarıdaki hücredeki yorum satırlarını kaldırın.")

---

## 📝 Özet

Bu notebook'ta şunları gerçekleştirdik:

1. **Mediapipe Pose** ile biceps curl videolarından pose landmark'ları çıkardık
2. Frame bazlı açı ve postür **feature'ları** hesapladık:
   - Dirsek açıları (min, max, range, mean, std)
   - Omuz stabilitesi (x/y koordinat varyasyonları)
   - Gövde açısı (torso stability)
   - Açısal hız (kinematik analiz)
3. Her video için bu feature'ları özetleyip tek bir feature vektörü oluşturduk
4. **RandomForestClassifier** ile model eğittik
5. Test set üzerinde performansı ölçtük:
   - Accuracy
   - Classification Report
   - Confusion Matrix
6. **Feature Importance** analizi yaptık
7. Modeli `.joblib` formatında kaydettik

---

### 🚀 İleriye Dönük İyileştirmeler

- [ ] Rep segmentasyonu ekleyerek her rep için ayrı analiz
- [ ] Cross-validation ile daha güvenilir performans tahmini
- [ ] Hyperparameter tuning (GridSearchCV)
- [ ] Farklı modeller deneme (XGBoost, LightGBM, Neural Network)
- [ ] Data augmentation (video rotation, flipping)

---

**🏋️ İyi antrenmanlar!**